# Arbitrary State Generation

## Step 1: Computing pulse sequences

In this first section, we develop tools for computing the pulse sequences necessary for generating arbitrary resonator states. We follow the algorithm laid out in the paper, starting with the desired final state and then repetitively emptying the top Fock state into the qubit, and proceeding down the ladder to the ground state. 

First, let's import modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qutip import *

Let's define some constants. 

In [2]:
t1q = 650  # qubit T1 in ns
t2q = 150  # qubit T2 in ns
t1r = 3500  # resonator T1 in ns
w_r = 2 * np.pi * 6.570  # resonator frequency in GHz
omega = 2 * np.pi * 19e-3  # qubit/cavity interaction strength in GHz
delta_on = 0  # qubit/cavity detuning when on resonance 
delta_off = - 2 * np.pi * 463e-3  # qubit/cavity detuning when off resonance in GHz

There are three types of pulses: swap operations, qubit drive operations, and phase rotation operations. Let's write some functions to calculate the length of time each step needs to be applied for. Let's start with the swap operation. In the rest of this section, we imagine we are descending the ladder, as this is the order in which the operations are computed. In each step, we focus only on the two highest-energy components of whatever the complete state is.

The swap operation needs to take a superposition $c_1 | e, n-1 \rangle + c_2 |g, n\rangle$ and transfer it to pure $|e, n-1\rangle$. When the qubit is tuned on resonance, we have Rabi oscillations between these two states at a frequency $\sqrt{n}\Omega$. We note that the previous rotation operation has left these two states in phase with one another, so we can assume that the coefficients $c_1$ and $c_2$ are real. Thus the on-resonance pulse duration is simply 
\begin{equation}
\tau_n = \frac{2}{\sqrt{n}\Omega}\sin^{-1} (c_2 / c_1 ).
\end{equation}
Let's make a function for this.

In [3]:
def swap_time(n, c1, c2):
    """Computes the on-resonance time necessary to swap c1 |e, n-1> + c2 |g, n> down to pure |e, n-1>."""
    return 2 * np.arcsin(c2 / c1) / omega / np.sqrt(n)

Next, let's deal with the phase rotation operations. In the swap operation, we assumed that the two swap states were in phase with each other; in general, this will not be true following a qubit drive operation. Thus we need to engineer this to be true, which is what the phase rotation is for. The phase rotation is accomplished by leaving the qubit detuned from the resonator at the detuning freuency $\Delta_\text{off}$, which causes the $|e, n-1\rangle$ states to accumulate phase at a rate of $\Delta_\text{off}$ relative to the $|g, n+1\rangle$ states. Thus, given the coefficients $c_1$ and $c_2$ for $|e, n-1\rangle$ and $|g, n+1\rangle$ (which can now be complex), we know that the time of the phase pulse is 
\begin{equation}
    t_n = \Delta_\text{off}\text{Arg} (c_1 / c_2).
\end{equation}
Here is a function for this:

In [4]:
def rotation_time(c1, c2, lb=0):
    """Computes the detuning time necessary to get c1 |e, n-1> in phase with c2 |g, n>.
    Includes an optional argument for a lower bound lb, in case you want to specify that this time must
    be longer than the qubit drive operation.
    """
    t = delta_off * np.angle(c1 / c2)
    while t < lb:
        t = t - 2 * np.pi * delta_off  # recall that delta_off is negative, hence the sign

Finally, we turn to the qubit drive operations, which are the most complicated. The qubit drive operations take place when the system is detuned and the two highest energy levels are $|e, n \rangle $ and $|g, n\rangle$. They need to take an arbitrary Bloch sphere vector $c_1 |e, n \rangle + c_2 |g, n\rangle$ and map it down to pure $|g, n\rangle$. The Hamiltonian in this situation is approximately
\begin{equation}
H = \Delta_\text{off} \sigma_+ \sigma_- + \frac{\Omega_q(t)}{2} \sigma_+ + \frac{\Omega_q^* (t)}{2} \sigma_- =
\Delta_\text{off} \sigma_+ \sigma_- + \text{Re} (\Omega_q (t)) \sigma^x + \text{Im}(\Omega_q(t)) \sigma^y,
\end{equation}
where the qubit/resonator interaction has been neglected because the detuning term is more important, and the cavity drive is turned off. We flip into the interaction picture with respect to the time-independent term of this Hamiltonian, and find that the ground state is unchanged while the excited state precesses at frequency $\Delta_\text{off}$. This corresponds to the Bloch sphere precessing around the $z$ axis. Thus, the simplest way to design the drive pulse $\Omega_q(t)$ is 
\begin{equation}
\Omega_q(t) = 
\begin{cases}
    0,  & \text{if } t < 0\\
    (a + i b) e^{-i\Delta_\text{off} t},  & \text{if } 0 \leq t \leq t_\text{max} \\
    0,  & \text{if } t > t_\text{max}
\end{cases}
\end{equation}
In the rotating frame, this is a constant pulse with complex amplitude $a + ib$ and duration $t_\text{max}$. The real part of the pulse will cause precession around the $x$ axis of the Bloch sphere and the imaginary part of the pulse will cause precession around the $y$ axis, so by picking these parameters correctly we can cause the state to rotate from an arbitrary location on the Bloch sphere down to pure $|g, n\rangle$. Specifically, we want to pick these parameters such that 
\begin{equation}
\text{Arg} (a + i b) = \frac{\pi}{2} + \text{Arg} (c_2 / c_1)
\end{equation}
and such that 
\begin{equation}
|a + ib| t_\text{max} = 2\sin^{-1} |c_2 / c_1 |.
\end{equation}
Note that we have a degree of freedom in the ratio of $t_\text{max}$ to $|a + ib|$. Let's pick $|a + ib| = \Delta_\text{off}$ so that this process is substantially stronger than the coupling term $\Omega$ (which is part of the approximation we made earlier when we dropped it). We can now write functions to compute these parameters:

In [9]:
def qubit_drive_phase(c1, c2):
    """Return the appropriate phase factor for the qubit drive pulse given a state c1 |e, n> + c2 |g, n>."""
    return np.pi / 2 + np.angle(c2 / c1)


def qubit_drive_time(c1, c2):
    """Return the appropriate time for the qubit drive pulse given a state c1 |e, n> + c2 |g, n> and 
    assuming the drive amplitude is Delta_off.
    """
    return 2 * np.arcsin(np.abs(c2 / c1)) / delta_off

Now we are to the point of working out the actual pulse sequences for the time dependent Hamiltonian. 

An important 